In [35]:
import pandas as pd

## Raw data cleaning and feature engineering

In [36]:
# Load in raw data from csv file
raw = pd.read_csv('../data/raw/adr6921_Suppl_Excel_v2.csv',
                  encoding_errors='replace')

In [37]:
# Inspect first few rows of the dataframe
raw.head(20)

,factory_id,country,lat,long,company,data_type,material_type,material_type_simple,comm_name,sci_name,func_group,source,website,date
0,1,Argentina,-37.9784,-57.5903,Agustiner S.A.,A,fishery by-products,by-products,NaN,NaN,NaN,IFFO,https://seafood.media/agustiner/,24-Feb
1,2,Argentina,-38.0462,-57.5459,Coomarpes Ltda.,A,fishery by-products,by-products,NaN,NaN,NaN,IFFO,https://www.coomarpes.com.ar/en/,24-Feb
2,3,Argentina,-42.7223,-65.0332,Harinas Patagonicas S.R.L,A,fishery by-products,by-products,shrimp,NaN,Shrimps,NaN,https://www.aquafeed.com/newsroom/news/argenti...,24-Apr
3,4,Argentina,-38.0015,-57.6648,Gihon- Laboratorios Químicos SRL,A,NaN,NaN,NaN,NaN,NaN,MarinTrust,https://www.gihonlab.com/en/naturales/producto...,24-Mar
4,5,Australia,-33.4335,150.9982,CSF Proteins Pty Ltd,A,misc by-products,by-products,NaN,NaN,NaN,IFFO,https://www.csfproteins.com.au,24-Feb
5,6,Australia,-37.8230,144.8131,CSF Proteins Pty Ltd,A,misc by-products,by-products,NaN,NaN,NaN,IFFO,https://www.csfproteins.com.au,24-Feb
6,7,Australia,-35.9502,150.0980,Proteins Australia Pty Ltd,A,whole,whole,mackerel,Scomber scombrus,Medium pelagics (30 - 89 cm),MarinTrust,NaN,24-Mar
7,8,Australia,-42.1581,146.6091,NuBlend Pty Ltd,C,misc by-products,by-products,NaN,NaN,NaN,https://www.trade-seafood.com/directory/seafoo...,https://www.nublend.com.au/index.php/products,24-Feb
8,9,Australia,-37.8158,144.9663,Australian Fishmeal Company,A,whole,whole,NaN,NaN,NaN,NaN,https://fishmeal.co/,24-Apr
9,10,Brazil,-29.5099,-52.0675,FASA,A,misc by-products,by-products,NaN,NaN,NaN,https://brazilianrenderers.com/members/,https://www.fasa.ind.br/produtos/?lang=en,24-Apr


In [38]:
# Aggregate data by country through pivot tables of different attributes
# Each pivot table counts the number of factories per country for each attribute category

# Total number of unique factories per country
factory_pivot = raw.pivot_table(
    index='country', values='factory_id', aggfunc='nunique')
# Number of factories per country per material type
material_pivot = raw.pivot_table(index='country', columns=[
                                 'material_type'], values='factory_id', aggfunc='nunique', fill_value=0)
# Number of factories per country per common name
comm_name_pivot = raw.pivot_table(index='country', columns=[
                                  'comm_name'], values='factory_id', aggfunc='nunique', fill_value=0)
# Number of factories per country per scientific name
sci_name_pivot = raw.pivot_table(index='country', columns=[
                                 'sci_name'], values='factory_id', aggfunc='nunique', fill_value=0)
# Number of factories per country per functional group
func_group_pivot = raw.pivot_table(index='country', columns=[
                                   'func_group'], values='factory_id', aggfunc='nunique', fill_value=0)

In [39]:
material_pivot.head()

material_type,aquaculture by-products,fishery by-products,misc by-products,whole
country,,,,
Argentina,0,3,0,0
Australia,0,0,3,2
Brazil,0,0,8,0
Canada,0,7,0,0
Chile,11,8,0,27


In [40]:
# Drop squid column from comm_name_pivot because that is present in func_group_pivot already and is just duplicated
comm_name_pivot = comm_name_pivot.drop(columns=['Squid'])

# Rename factory_id column in factory_pivot to total_factories for clarity
factory_pivot = factory_pivot.rename(columns={'factory_id': 'total_factories'})

In [41]:
# Obtain list of unique countries in dataset and set it as dataframe index
countries = raw['country'].unique()
countries_df = pd.DataFrame(index=countries)
countries_df.index.name = 'country'

In [42]:
# Join all pivot tables into a final dataframe indexed by country
final = countries_df.join(
    [factory_pivot, material_pivot, comm_name_pivot, sci_name_pivot, func_group_pivot])

# reset index to make country a column again
final = final.reset_index()

# Fill Nan values with 0
final = final.fillna(0)

# Write final dataframe to csv
final.to_csv('../data/processed/factory_counts_by_country.csv', index=False)

In [43]:
# Inspect first few rows of the final dataframe
final.head()

,country,total_factories,aquaculture by-products,fishery by-products,misc by-products,whole,Alaska Pollack,Albacore Tuna,Anchoveta,Anchovy,...,Medium reef assoc. fish (30 - 89 cm),Other demersal invertebrates,Shrimps,Small benthopelagics (<30 cm),Small demersals (<30 cm),Small pelagics (<30 cm),Small reef assoc. fish (<30 cm),Small to medium flatfishes (<90 cm),Small to medium rays (<90 cm),Squid
0,Argentina,4,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Australia,5,0.0,0.0,3.0,2.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Brazil,8,0.0,0.0,8.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Canada,7,0.0,7.0,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0
4,Chile,33,11.0,8.0,0.0,27.0,0.0,0.0,4.0,8.0,...,0.0,0.0,1.0,0.0,0.0,23.0,0.0,0.0,0.0,0.0


In [44]:
# Inspect data for Chile and Peru
final[(final['country'] == 'Chile') | (final['country'] == 'Peru')]

,country,total_factories,aquaculture by-products,fishery by-products,misc by-products,whole,Alaska Pollack,Albacore Tuna,Anchoveta,Anchovy,...,Medium reef assoc. fish (30 - 89 cm),Other demersal invertebrates,Shrimps,Small benthopelagics (<30 cm),Small demersals (<30 cm),Small pelagics (<30 cm),Small reef assoc. fish (<30 cm),Small to medium flatfishes (<90 cm),Small to medium rays (<90 cm),Squid
4,Chile,33,11.0,8.0,0.0,27.0,0.0,0.0,4.0,8.0,...,0.0,0.0,1.0,0.0,0.0,23.0,0.0,0.0,0.0,0.0
40,Peru,125,0.0,49.0,0.0,76.0,0.0,0.0,76.0,0.0,...,0.0,0.0,0.0,0.0,0.0,76.0,0.0,0.0,0.0,0.0


In [45]:
# Convert columns with counts the number of factories per country for each attribute category into percent of total factories in that country for better comparison between countries
final_percent = final.copy()
# Skip first two columns (country and total_factories)
for col in final.columns[2:]:
    final_percent[col] = round(final[col] / final['total_factories']*100, 2)

In [46]:
final_percent.head()

,country,total_factories,aquaculture by-products,fishery by-products,misc by-products,whole,Alaska Pollack,Albacore Tuna,Anchoveta,Anchovy,...,Medium reef assoc. fish (30 - 89 cm),Other demersal invertebrates,Shrimps,Small benthopelagics (<30 cm),Small demersals (<30 cm),Small pelagics (<30 cm),Small reef assoc. fish (<30 cm),Small to medium flatfishes (<90 cm),Small to medium rays (<90 cm),Squid
0,Argentina,4,0.00,75.00,0.0,0.00,0.0,0.00,0.00,0.00,...,0.0,0.0,25.00,0.0,0.00,0.00,0.0,0.0,0.0,0.0
1,Australia,5,0.00,0.00,60.0,40.00,0.0,0.00,0.00,0.00,...,0.0,0.0,0.00,0.0,0.00,0.00,0.0,0.0,0.0,0.0
2,Brazil,8,0.00,0.00,100.0,0.00,0.0,0.00,0.00,0.00,...,0.0,0.0,0.00,0.0,0.00,0.00,0.0,0.0,0.0,0.0
3,Canada,7,0.00,100.00,0.0,0.00,0.0,14.29,0.00,14.29,...,0.0,0.0,0.00,0.0,14.29,28.57,0.0,0.0,0.0,0.0
4,Chile,33,33.33,24.24,0.0,81.82,0.0,0.00,12.12,24.24,...,0.0,0.0,3.03,0.0,0.00,69.70,0.0,0.0,0.0,0.0


In [47]:
# Making sure none of the rows have a ratio more than 1
final_percent[(final_percent[final_percent.columns[2:]] < 75).all(axis=1)]

,country,total_factories,aquaculture by-products,fishery by-products,misc by-products,whole,Alaska Pollack,Albacore Tuna,Anchoveta,Anchovy,...,Medium reef assoc. fish (30 - 89 cm),Other demersal invertebrates,Shrimps,Small benthopelagics (<30 cm),Small demersals (<30 cm),Small pelagics (<30 cm),Small reef assoc. fish (<30 cm),Small to medium flatfishes (<90 cm),Small to medium rays (<90 cm),Squid
1,Australia,5,0.00,0.00,60.0,40.00,0.00,0.0,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00
5,China,22,0.00,0.00,0.0,0.00,0.00,0.0,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00
8,Ecuador,18,0.00,72.22,0.0,27.78,0.00,0.0,0.00,0.00,...,0.00,0.00,0.00,5.56,0.00,72.22,0.00,0.00,0.0,0.00
13,France,6,50.00,33.33,0.0,33.33,0.00,0.0,0.00,16.67,...,0.00,16.67,0.00,0.00,0.00,50.00,0.00,33.33,0.0,16.67
16,Germany,3,0.00,33.33,0.0,33.33,0.00,0.0,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,33.33,0.00,0.00,0.0,0.00
18,India,23,0.00,17.39,0.0,43.48,0.00,0.0,0.00,4.35,...,4.35,0.00,0.00,0.00,8.70,39.13,4.35,0.00,0.0,0.00
19,Indonesia,11,0.00,45.45,0.0,36.36,0.00,0.0,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,36.36,0.00,0.00,0.0,0.00
20,Iran,3,0.00,33.33,0.0,33.33,0.00,0.0,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,33.33,0.00,0.00,0.0,0.00
21,Ireland,3,33.33,66.67,0.0,66.67,0.00,0.0,0.00,33.33,...,0.00,0.00,0.00,0.00,33.33,66.67,0.00,0.00,0.0,0.00
22,Japan,3,0.00,0.00,0.0,0.00,0.00,0.0,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00


## Clustering algorithm

- Standardizing
- Dimension reduction (PCA? NMF?)
- Model selection: Hierarchial

In [90]:
X = final_percent.drop(['country','total_factories'], axis=1)

In [91]:
X

,aquaculture by-products,fishery by-products,misc by-products,whole,Alaska Pollack,Albacore Tuna,Anchoveta,Anchovy,Anglerfish,Araucanian Herring,...,Medium reef assoc. fish (30 - 89 cm),Other demersal invertebrates,Shrimps,Small benthopelagics (<30 cm),Small demersals (<30 cm),Small pelagics (<30 cm),Small reef assoc. fish (<30 cm),Small to medium flatfishes (<90 cm),Small to medium rays (<90 cm),Squid
0,0.00,75.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,25.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,0.00,0.00,60.0,40.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2,0.00,0.00,100.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3,0.00,100.00,0.0,0.00,0.00,14.29,0.00,14.29,0.00,0.00,...,0.00,0.00,0.00,0.00,14.29,28.57,0.00,0.00,0.00,0.00
4,33.33,24.24,0.0,81.82,0.00,0.00,12.12,24.24,0.00,15.15,...,0.00,0.00,3.03,0.00,0.00,69.70,0.00,0.00,0.00,0.00
5,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
6,0.00,100.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,100.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
7,20.00,100.00,0.0,80.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,80.00,80.00,80.00,0.00,80.00,0.00,0.00
8,0.00,72.22,0.0,27.78,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,5.56,0.00,72.22,0.00,0.00,0.00,0.00
9,0.00,100.00,0.0,0.00,0.00,100.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [133]:
from sklearn.decomposition import NMF
k=5

nmf = NMF(n_components=k, init="nndsvda", random_state=0)
W = nmf.fit_transform(X)   # countries × components
H = nmf.components_       # components × categories

In [134]:
countries = final_percent["country"]
categories = X.columns

# W (country mixtures)
W_df = pd.DataFrame(W,
                    index=countries,
                    columns=[f"Type {i+1}" for i in range(k)])

# Normalize W to percentages
W_df = W_df.div(W_df.sum(axis=1), axis=0)

# H (component definitions)
H_df = pd.DataFrame(H,
                    columns=categories,
                    index=[f"Type {i+1}" for i in range(k)])

In [137]:
W_df

,Type 1,Type 2,Type 3,Type 4,Type 5
country,,,,,
Argentina,0.000000,0.075656,0.000000,0.669895,0.254449
Australia,0.000000,0.000000,0.766509,0.000000,0.233491
Brazil,0.000000,0.000000,1.000000,0.000000,0.000000
Canada,0.000000,0.134699,0.161376,0.349628,0.354297
Chile,0.000000,0.000382,0.658498,0.008127,0.332993
China,NaN,NaN,NaN,NaN,NaN
Costa Rica,0.000000,0.045299,0.000000,0.735211,0.219490
Denmark,0.999301,0.000699,0.000000,0.000000,0.000000
Ecuador,0.000000,0.497208,0.392405,0.110387,0.000000


In [ ]:
import altair as alt

H_long = H_df.reset_index().melt(
    id_vars="index",
    var_name="Category",
    value_name="Weight"
)

H_long = H_long.rename(columns={"index": "Component"})

bars = alt.Chart(H_long).mark_bar().encode(
    x=alt.X("Weight:Q", title="Contribution"),
    y=alt.Y("Category:N", sort="-x"),
    tooltip=["Component", "Category", "Weight"]
).facet(
    row=alt.Row("Component:N", title="NMF Components")
).resolve_scale(
    y="independent"
)

bars

alt.FacetChart(...)